<a href="https://colab.research.google.com/github/kaz2ngt/colaboratory/blob/master/12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備: ドライブのマウントとデータの取得

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# 下記でdriveからコピーして解凍
#!cp drive/My\ Drive/Colab\ Notebooks/12data.zip ./
#!unzip 12data.zip

# リストPART2

In [0]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

In [0]:
# 足りないimport追加
from tensorflow.python.keras.layers import Flatten, Reshape

# 12_1: 画像データの読み込み

In [0]:
DATA_DIR = 'data/chap12/'
BATCH_SIZE = 16
IMG_SHAPE = (64, 64, 3)

data_gen = ImageDataGenerator(rescale=1/255.)
train_data_generator = data_gen.flow_from_directory(
    directory=DATA_DIR,
    classes=['faces'],
    class_mode=None,
    batch_size=BATCH_SIZE,
    target_size=IMG_SHAPE[:2]
)

Found 19370 images belonging to 1 classes.


# 12_2: Encoderの定義

In [0]:
def build_encoder(input_shape, z_size, n_filters, n_layers):
  """Encoderを構築する

  Arguments:
    input_shape (int): 画像のshape
    z_size (int): 特徴空間の次元数
    n_filters (int): フィルタ数

  Returns:
   model (Model): Encoderモデル
  """
  model = Sequential()
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          input_shape=input_shape,
          padding='same'
      )
  )
  model.add(Conv2D(n_filters, 3, padding='same'))
  for i in range(2, n_layers + 1):
    model.add(
        Conv2D(
            i*n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(
        Conv2D(
            i*n_filters,
            3,
            activation='elu',
            strides=2,
            padding='same'
        )
    )
  model.add(Conv2D(n_layers*n_filters, 3, padding='same'))
  model.add(Flatten())
  model.add(Dense(z_size))

  return model

# 12_3: Generator/Decoderの定義

In [0]:
def build_decoder(output_shape, z_size, n_filters, n_layers):
  """Decoderを構築する

  Arguments:
    output_shape (np.array): 画像のshape
    z_site (int): 特徴空間の次元数
    n_filters (int): フィルタ数
    n_layers (int): レイヤー数

  Returns:
    model (Model):  Decoderモデル
  """
  # UpSampling2Dで何倍に拡大されるか
  scale = 2**(n_layers - 1)
  # 最初の畳み込み層の入力サイズをscaleから逆算
  fc_shape = (
      output_shape[0]//scale,
      output_shape[1]//scale,
      n_filters
 )
  # 全結合層で必要なサイズを逆算
  fc_size = fc_shape[0]*fc_shape[1]*fc_shape[2]

  model = Sequential()
  # 全結合層
  model.add(Dense(fc_size, input_shape=(z_size,)))
  model.add(Reshape(fc_shape))

  # 畳み込み層の繰り返し
  for i in range(n_layers - 1):
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(UpSampling2D())

  # 最後の層はUpSampling2Dが不要
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          padding='same',
      )
  )
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          padding='same'
      )
  )
  # 出力層で3チャンネルに
  model.add(Conv2D(3, 3, padding='same'))

  return model

# 12_4: Generatorの定義

In [0]:
def build_generator(img_shape, z_size, n_filters, n_layers):
  decoder = build_decoder(
      img_shape, z_size, n_filters, n_layers
  )
  return decoder

# 12_5: Discriminatorの定義

In [0]:
def build_discriminator(img_shape, z_size, n_filters, n_layers):
  encoder = build_encoder(
      img_shape, z_size, n_filters, n_layers
  )
  decoder = build_decoder(
      img_shape, z_size, n_filters, n_layers
  )
  return Sequential((encoder, decoder))

# 12_6: Discriminatorの学習用のネットワーク

In [0]:
def build_discriminator_trainer(discriminator):
  img_shape = discriminator.input_shape[1:]
  real_inputs = Input(img_shape)
  fake_inputs = Input(img_shape)
  real_outputs = discriminator(real_inputs)
  fake_outputs = discriminator(fake_inputs)

  return Model(
      inputs=[real_inputs, fake_inputs],
      outputs=[real_outputs, fake_outputs]
  )

# 12_7: ネットワークの構築

In [57]:
n_filters = 64 # フィルタ数
n_layers = 4# レイヤー数
z_size = 32 # 特徴空間の次元数

generator = build_generator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator = build_discriminator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator_trainer  =  build_discriminator_trainer(
    discriminator
)

generator.summary()
# discriminator.layers[1]が Decoder を表す
discriminator.layers[1].summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 4096)              135168    
_________________________________________________________________
reshape_22 (Reshape)         (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_306 (Conv2D)          (None, 8, 8, 64)          36928     
_________________________________________________________________
conv2d_307 (Conv2D)          (None, 8, 8, 64)          36928     
_________________________________________________________________
up_sampling2d_66 (UpSampling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_308 (Conv2D)          (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_309 (Conv2D)          (None, 16, 16, 64)      